In [1]:
import numpy as np
import pandas as pd

In [2]:
rg = np.random.default_rng(0)

sizes = np.array([round(i,4) for i in rg.random((1, 9))[0]])
prices = np.linspace(1, 1.3, 9)
book = np.array([sizes,prices]).T
book_df = pd.DataFrame(book,columns = ['size','price'])
vwap_depth = 5.067
# VWAP_SIZE = book[0:3,0].sum()

In [3]:
book

array([[0.637 , 1.    ],
       [0.2698, 1.0375],
       [0.041 , 1.075 ],
       [0.0165, 1.1125],
       [0.8133, 1.15  ],
       [0.9128, 1.1875],
       [0.6066, 1.225 ],
       [0.7295, 1.2625],
       [0.5436, 1.3   ]])

In [240]:
def get_vwap(book,vwap_depth):
    vwap_lb = book[book[:,0].cumsum() < vwap_depth]
    vwap_ub = book[book[:,0].cumsum() >= vwap_depth]
    if vwap_ub.size ==0: # vwap could not be resolved due to insufficient depth
        return np.nan
    vwap_lb_val = np.dot(vwap_lb[:,0],vwap_lb[:,1])
    vwap_ub_val = (vwap_depth - vwap_lb[:,0].sum()) * vwap_ub[0,1]
    vwap = (vwap_lb_val + vwap_ub_val)/vwap_depth
    return vwap

In [261]:
vwap_lb = book[book[:,0].cumsum() < vwap_depth]
vwap_ub = book[book[:,0].cumsum() >= vwap_depth]
if vwap_ub.size ==0: # vwap could not be resolved due to insufficient depth
    pass
vwap_lb_val = np.dot(vwap_lb[:,0],vwap_lb[:,1])
vwap_ub_val = (vwap_depth - vwap_lb[:,0].sum()) * vwap_ub[0,1]
vwap = (vwap_lb_val + vwap_ub_val)/vwap_depth

In [255]:
book

array([[0.637 , 1.    ],
       [0.2698, 1.0375],
       [0.041 , 1.075 ],
       [0.0165, 1.1125],
       [0.8133, 1.15  ],
       [0.9128, 1.1875],
       [0.6066, 1.225 ],
       [0.7295, 1.2625],
       [0.5436, 1.3   ]])

In [241]:
book_df['size']

0    0.6370
1    0.2698
2    0.0410
3    0.0165
4    0.8133
5    0.9128
6    0.6066
7    0.7295
8    0.5436
Name: size, dtype: float64

In [275]:
def get_df_vwap(book_df,vwap_depth):
    vwap_lb = book_df[book_df['size'].cumsum() < vwap_depth]
    vwap_ub = book_df[book_df['size'].cumsum() >= vwap_depth]
    if vwap_ub.empty: # vwap could not be resolved due to insufficient depth
        return np.nan
    vwap_lb_val = (vwap_lb['size'] * vwap_lb['price']).sum()
    vwap_ub_val = (vwap_depth - vwap_lb['size'].sum()) * vwap_ub.iloc[0].price
    vwap = (vwap_lb_val + vwap_ub_val)/vwap_depth
    return vwap

In [286]:
assert get_df_vwap(book_df,3) == get_vwap(book,3)
assert get_df_vwap(book_df,1) == get_vwap(book,1)
assert np.isnan(get_df_vwap(book_df,8)) == np.isnan(get_vwap(book,8))

In [308]:
%%timeit 
get_vwap(book,3)

16.5 µs ± 593 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [298]:
%%timeit 
get_vwap(book,7)

9.17 µs ± 214 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [307]:
%%timeit 
get_df_vwap(book_df,3)

1.07 ms ± 7.83 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [309]:
%%timeit
get_df_vwap(book_df,7)

716 µs ± 11.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


* np version of get_vwap runs in 16.5 µs (microsec) (9.2 µs if insufficient depth)
* np version of get_vwap runs in 1.07 ms (millisec) (716 µs if insufficient depth)
* around 60 times faster